"Who needs friends? Zanzil just makes his own!"

친구가 없으면 어때? 잔질처럼 이렇게 만들면 되는데!  

WOW Zul'Gurub Zanzil

와우 줄구룹 잔질, 의술사

### 클래스 목록
- AI플레이어 (Player)
- 닝겐 유저 (User)
- Environment (as env)
- gamefunctions (as gf)

### 설정 값
- 게임 룰 세팅 (일단 'default'는 충곽5마 룰에 따라 진행함. 기억과 장작위키에 의존...)
- 유저 수 (기본 5마)

### 플레이에 필요한 기능
- 카드 돌리기 (구현)
- 딜미스 판단 (구현)
- 공약 (rule base 구현)
- 기루다, 조커콜, 마이티 선언 (구현)
- 주공이 프랜드 선택 (rule base 구현)
- 플레이

### 초반 구현 필요한 부분
- 룰대로 게임이 돌아가는것 (규칙 안 어기면서 카드 내기 가능하게) (구현)
- AI개발모드(플레이어 없이/있이 플레이 가능하게) (설정 가능, 구현)

### 중반 구현 필요
- AI, 인터페이스, 그래픽, 변형룰 옵션, 누적 점수 계산

### 후반 구현
- 앱으로 만들기 (2월까지 될까...?)
- 플레이스토어 등록 (시작은 원대하나 그 끝은..?)

### 최후반 옵션
- 리플레이 저장하면 어떨까?
    - 어케 해야 마이티 잘하는지 컴한테 배우게 ㅋㅋㅋㅋ

In [ ]:
# from gamefunctions import *

In [2]:
import gamefunctions as gf
from player import Player
from user import User
from environment import Environment

import re, random

## 진행

In [18]:
env = Environment() #게임의 진행을 저장. 플레이어들이 확인할 수 있는 정보들
playerNumber, rule = gf.ruleSetting(ruleName = 'AIs')#ruleName = 'AIs' # 룰 세팅

deck = gf.deckGenerator() # 덱 생성
grave = [] # 무덤 선언

### main

# 룰에 따른 플레이어 수 맞추기
if rule == "defalt":
    k = 1
elif rule == "AIs":
    k = 0
else: print('!!! Setting Error !!!')

# 플레이어 생성
players = list(range(0,playerNumber))
playerName = ['윤희', '상훈', '예찬', '광호', '길용', '루피']
# random.shuffle(playerName)
for i in range(playerNumber-k):
    players[i] = Player()
    players[i].name = playerName[i]
#     print(players[i].name) #체크포인트, cp, 사실 별 필요 없음.
if k == 1:
    players[playerNumber-1] = User() #유저(닝겐)은 특별취급~!
    players[playerNumber-1].name = playerName[playerNumber-1]
#     print(players[playerNumber-1].name)

# 포지션을 정한다...
# position = [0,1,2,3,4]
turn = list(range(0,playerNumber)) #턴(포지션으로 생각해도됨), 게임을 끝내고 새게임 시작할때 바꿀 수 있을.
random.shuffle(turn)
env.locations = turn

In [24]:
# 카드 나눠주기 & 딜미스 판단
mighty, jokerCall = gf.setMightyJokercall()
joker = ['joker', 0]

#role initialize
for i in range(env.playerNumber):
    players[i].role = [0,0]
    players[i].score = 0
    players[i].grave = []


missCount = playerNumber #딜미스 카운트
while missCount>0:
    grave = gf.handout(players, deck, grave)
    missCount = playerNumber
    for i in range(playerNumber):
        if gf.dealCount(players[i].hand) >=2: missCount -=1
#     if k ==1:
#         if dealCount(user.hand)>=2: missCount -=1
#     print(missCount)
    if missCount>0: print('Miss!')
print(grave)


# 주공 공약
biddingCount = playerNumber
turnCount = 0
others = ["",0]
preOthers = ["",11]
kingNo = 0

# for i in turn:
Jugong = 0 #주공: players[turn[Jugong]]
turnStart = 0 #매 턴 시작을 누가하는지 (첫턴엔 주공, 그다음부턴 이긴사람)
while biddingCount >0:
    others = players[turn[turnCount]].pledge(preOthers) #인자로 다른애들 공약을 넣어야 
    print("")
    adjustment = 0
    if others[0] == 'noGiru':
        adjustment = 1
    if others[1]>preOthers[1] - adjustment:
        preOthers = others
        kingNo = turn[turnCount]
        biddingCount = playerNumber-1
        
    else: biddingCount -=1
    print(players[turn[turnCount]].name+"'s bidding is: "+str(others[0]) + ", " +str(others[1]))
    
    # 원형 큐 처럼 반복되게 하는거... 나는 바보같은 방법밖에 생각안나네 ㅠ
    if turnCount < playerNumber-1:
        turnCount+= 1
    else: turnCount = 0
    
    if biddingCount == 0 :
        players[turn[turnCount]].role = [1,0]
        turnStart = turnCount
    #순서대로 공약을 한다.
        # 공약이 안 나오면 카운트를 -1 한다
        # 공약이 나오면 카운트를 playerNumber-1 한다
        #계속 반복한다

Miss!
Miss!
Miss!
[['H', 6], ['D', 3], ['H', 9]]

예찬's bidding is: , 1

윤희's bidding is: , 1

상훈's bidding is: , 1

길용's bidding is: C, 12

광호's bidding is: , 1

예찬's bidding is: , 1

윤희's bidding is: , 1

상훈's bidding is: , 1


In [25]:
# 마이티: SA, S가 기루다일 땐 DA(빨마,빨강마이티)
# 조커콜: C3, C가 기루다일 땐 S3(관악, 경기에선 H3)
giruda = preOthers[0]
env.giruda = giruda
pledge = preOthers[1]
env.bidding = pledge
players[kingNo].role = [1,0]   #00:야당, 01:프랜드, 10: 주공
env.setDeclarer(kingNo)
if giruda == "S": env.mighty = mighty = ['D', 14]
else: mighty = ['S', 14]
if giruda == "C": env.jokerCall = jokercall = ['H', 3]
else: jokerCall = ['C', 3]    

# print('preOthers is ' + str(preOthers))

# print('kingNO is '+str(kingNo))
# print("")

# print('hand is:')
# print(sorted(players[kingNo].hand))
# print('')

# print('grave is: ')
# print(grave)

grave = players[kingNo].kingGrave(grave, giruda)


# print(sorted(players[kingNo].hand))
# print(grave)



## callFriend
friendCard = players[kingNo].callFriend(giruda)
print(friendCard)
for i in turn: #프랜드는 본인이 프랜드인걸 알고 플레이 해야 해
    if friendCard in players[i].hand:
        players[i].role = [0,1]
#         break

# friendNo = players[kingNo].callFriend()
# players[friendNo].role = [0,1]

giru delete: C
['C', 4]
['C', 12]
['C', 7]
['C', 14]
['C', 8]
['S', 14]


### From now on
첫 라운드 
 - 조커 힘 약함

매 라운드 
- 핸드 1 이상인동안 반복
- 조커 내도 됨

- 선카 나오는거 보고 turnShape 정하기
- 조커를 첫카드로 낼 때는 무슨 카드를 선카로 할 지 선택
- player 클래스 내에 possibleCards 함수 만들기. 그 중에서 고르도록
- pickCard 함수에서는 그 중에 랜덤하게 내기
- 라운드에 나온 카드가 뭐였는지 기록
- 라운드 끝날 때는 무덤에 라운드 카드 넣기
- 무덤이 2개가 되야할듯. 1개는 kingGrave(버린 카드 3개 더 안다 남보다), 다른건 normalGrave


In [26]:
# only king knows what cards are in initial grave.
# graveKing = grave.copy()
# graveNormal = grave.copy()

roundNo = 0
# turnShape = "all" #첨엔 아무거나 픽 가능
env.roundShape = 'all'
roundHistory = []
# 누가 뭘 냈는지를 기억해야 하는데, 걍 리스트로 던져주면 어떻게 알지....
# 뭔가 방법이 필요하다.
# 매 턴 나온 카드를 53비트로 수정해서 넣어줘야 하나...?

position = turn

In [27]:
giruda

'C'

In [28]:
# first round
# 줄줄이 짜는게 좋냐 함수로 만들어서 콜하는게 좋냐
# 조커콜이 첫 카드면 조커콜 처리 함수 불러오자
# 일단 짜보자

"""
첫번째 사람이 카드내기: 첫턴 아무거나 낼 수 있다 - (첫턴 한정)기루 힘 없고 조커 힘 없다
    낸 카드에 대해서 turnShape 정해진다
    조커콜인지 확인
"""

# env.initialize() #이니셜라이즈 시키는건 있다가 다시보자 정신 맑을때.... 이거땜에 자꾸 에러난다.
env.scores = [0,0,0,0,0]
# env.giruda = giruda
# if giruda == "S": env.mighty = mighty = ['D', 14]
# else: mighty = ['S', 14]
# if giruda == "C": env.jokerCall = jokercall = ['H', 3]
# else: jokerCall = ['C', 3]  

env.roundTrun = gf.turnSetting(position, kingNo) #첫 턴은 kingNo에서 시작한다.
env.roundShape = 'all' #매 턴 시작은 '선'이 all이다.

env.role = [0,0,0,0,0]
env.role[env.locations.index(kingNo)] = 1
# players[env.roundTrun[0]].grave.append(grave) #king만 초기 grave를 알고있다.
players[env.roundTrun[0]].grave = grave.copy()


env.roundCards[0] = players[kingNo].pickCard(env.bits, env.roundShape, env.mighty) #말고도 giruda, round, history, score, 
if env.roundCards[0] != joker:
    env.roundShape = env.roundCards[0][0]
else:
    env.roundShape = players[kingNo].sayShape()

print(players[env.roundTrun[0]].name + "'s first card is "+ str(env.roundCards[0]))
if env.roundCards[0] == friendCard:
    print(players[env.roundTrun[0]].name + " is the Friend!!!!")
    env.role[env.locations.index(env.roundTrun[0])] = 1

    
if env.roundCards[0] != env.jokerCall: #노말턴, 조커콜 아님
    for i in range(1,env.playerNumber):
        env.roundCards[i] = players[env.roundTrun[i]].pickCard(env.bits, env.roundShape, env.mighty)
        print(players[env.roundTrun[i]].name + "'s choise is " + str(env.roundCards[i]))
        if env.roundCards[i] == friendCard:
            print(players[env.roundTrun[i]].name + " is the Friend!!!!")
            env.role[env.locations.index(env.roundTrun[i])] = 1 # 프랜드 정체 등록
else:
    for i in range(1,env.playerNumber):
        players[env.roundTrun[i]].jokerCallPick(env.bits, env.roundShape, env.mighty)

# for i in range(1,env.playerNumber):
    
winIndex, roundScore = gf.roundWin(env.roundCards, env.mighty, env.giruda, env.jokerCall, 0)
print('player '+players[env.roundTrun[winIndex]].name + ' win first round')

for i in players:
    i.grave += env.roundCards  # 무덤에 합쳐주고
env.roundCards = list(range(0,env.playerNumber)) #roundCards 초기화

players[env.roundTrun[winIndex]].score += roundScore
env.scores[env.locations.index(env.roundTrun[winIndex])] += roundScore  #env에 점수 추가...
nextTurnStart = env.roundTrun[winIndex]

"""
이긴사람: env.roundTrun[winIndex]
여기서부터 시작해야 해.

이긴사람 스코어 더해주고

나왔던 카드는 grave에 더해야 해.
"""

    
    
# 다음 순서가 카드를 낸다. 이때 입력 정보는 env와 grave가 갖고있다. king과 non-king은 각기 다른 grave를 가진다. 낸건 env.roundCards[]가 갖는다.
#     (grave는 player 각각이 갖고있어야 할듯...)
# 각 카드가 나올 때마다 friend 카드인지 확인한다.

#     - 첫턴은 "기루랑" 조커가 힘이 없다.
# 라운드 턴은 env.history에 저장한다. 카드 이름으로. 리플레이 볼 수 있도록.
# grave도 이걸 더한다. 



"""
    고민... 자리에 대한 정보를 어떻게 처리할지
    고민... 각 턴에 나온 히스토리를 어떻게 처리할지...
"""


"""
결과 판단:
    다음턴을 위해 순서를...
자리에 대한 정보를 어떻게 tf에 넣어줄지도 고민이다....
"""


길용's first card is ['D', 10]
광호's choise is ['D', 2]
예찬's choise is ['D', 8]
윤희's choise is ['D', 14]
상훈's choise is ['S', 14]
상훈 is the Friend!!!!
player 상훈 win first round


'\n결과 판단:\n    다음턴을 위해 순서를...\n자리에 대한 정보를 어떻게 tf에 넣어줄지도 고민이다....\n'

In [29]:
while len(players[0].hand) > 1:
    env.roundTrun = gf.turnSetting(position, nextTurnStart) #첫 턴은 kingNo에서 시작한다.

    env.roundCards[0] = players[env.roundTrun[0]].pickCard(env.bits, env.roundShape, env.mighty) #말고도 giruda, round, history, score, 
    if env.roundCards[0] != joker:
        env.roundShape = env.roundCards[0][0]
    else:
        env.roundShape = players[kingNo].sayShape()

    print(players[env.roundTrun[0]].name + "'s first card is "+ str(env.roundCards[0]))
    if env.roundCards[0] == friendCard:
        print(players[env.roundTrun[0]].name + " is the Friend!!!!")
        env.role[env.locations.index(env.roundTrun[0])] = 1

    if env.roundCards[0] != env.jokerCall: #노말턴, 조커콜 아님
        for i in range(1,env.playerNumber):
            env.roundCards[i] = players[env.roundTrun[i]].pickCard(env.bits, env.roundShape, env.mighty)
            print(players[env.roundTrun[i]].name + "'s choise is " + str(env.roundCards[i]))
            if env.roundCards[i] == friendCard:
                print(players[env.roundTrun[i]].name + " is the Friend!!!!")
                env.role[env.locations.index(env.roundTrun[i])] = 1
    else:
        for i in range(1,env.playerNumber):
            env.roundCards[i] = players[env.roundTrun[i]].jokerCallPick(env.bits, env.roundShape, env.mighty)
            print(players[env.roundTrun[i]].name + "'s choise is " + str(env.roundCards[i]))
            if env.roundCards[i] == friendCard:
                print(players[env.roundTrun[i]].name + " is the Friend!!!!")
                env.role[env.locations.index(env.roundTrun[i])] = 1

    # for i in range(1,env.playerNumber):

    winIndex, roundScore = gf.roundWin(env.roundCards, env.mighty, env.giruda, env.jokerCall, 1)
    print('player '+players[env.roundTrun[winIndex]].name + ' win this round')
    
    for i in players:
        i.grave += env.roundCards  # 무덤에 합쳐주고
    env.roundCards = list(range(0,env.playerNumber)) #roundCards 초기화

    players[env.roundTrun[winIndex]].score += roundScore
    env.scores[env.locations.index(env.roundTrun[winIndex])] += roundScore
    nextTurnStart = env.roundTrun[winIndex]
    print('turn end')
    print()

상훈's first card is ['D', 11]
길용's choise is ['D', 13]
광호's choise is ['D', 5]
예찬's choise is ['S', 2]
윤희's choise is ['joker', 0]
player 윤희 win this round
turn end

윤희's first card is ['D', 6]
상훈's choise is ['D', 4]
길용's choise is ['D', 12]
광호's choise is ['D', 9]
예찬's choise is ['C', 11]
player 예찬 win this round
turn end

예찬's first card is ['S', 12]
윤희's choise is ['S', 9]
상훈's choise is ['S', 8]
길용's choise is ['S', 7]
광호's choise is ['S', 11]
player 예찬 win this round
turn end

예찬's first card is ['S', 3]
윤희's choise is ['H', 11]
상훈's choise is ['S', 10]
길용's choise is ['C', 8]
광호's choise is ['S', 4]
player 길용 win this round
turn end

길용's first card is ['C', 7]
광호's choise is ['H', 12]
예찬's choise is ['C', 3]
윤희's choise is ['C', 2]
상훈's choise is ['C', 9]
player 상훈 win this round
turn end

상훈's first card is ['C', 13]
길용's choise is ['C', 12]
광호's choise is ['S', 5]
예찬's choise is ['C', 6]
윤희's choise is ['H', 13]
player 상훈 win this round
turn end

상훈's first card is ['C', 10]
길

In [30]:
env.roundTrun = gf.turnSetting(position, nextTurnStart) #첫 턴은 kingNo에서 시작한다.

env.roundCards[0] = players[env.roundTrun[0]].pickCard(env.bits, env.roundShape, env.mighty) #말고도 giruda, round, history, score, 
if env.roundCards[0] != joker:
    env.roundShape = env.roundCards[0][0]
else:
    env.roundShape = players[kingNo].sayShape()

print(players[env.roundTrun[0]].name + "'s first card is "+ str(env.roundCards[0]))
if env.roundCards[0] == friendCard:
    print(players[env.roundTrun[0]].name + " is the Friend!!!!")
    env.role[env.locations.index(env.roundTrun[0])] = 1


if env.roundCards[0] != env.jokerCall: #노말턴, 조커콜 아님
    for i in range(1,env.playerNumber):
        env.roundCards[i] = players[env.roundTrun[i]].pickCard(env.bits, env.roundShape, env.mighty)
        print(players[env.roundTrun[i]].name + "'s choise is " + str(env.roundCards[i]))
        if env.roundCards[i] == friendCard:
            print(players[env.roundTrun[i]].name + " is the Friend!!!!")
            env.role[env.locations.index(env.roundTrun[i])] = 1
else:
    for i in range(1,env.playerNumber):
        players[env.roundTrun[i]].jokerCallPick(env.bits, env.roundShape, env.mighty)

# for i in range(1,env.playerNumber):

winIndex, roundScore = gf.roundWin(env.roundCards, env.mighty, env.giruda, env.jokerCall, 0)
print('player '+players[env.roundTrun[winIndex]].name + ' win final round')

for i in players:
    i.grave += env.roundCards  # 무덤에 합쳐주고
env.roundCards = list(range(0,env.playerNumber)) #roundCards 초기화

players[env.roundTrun[winIndex]].score += roundScore
env.scores[env.locations.index(env.roundTrun[winIndex])] += roundScore
nextTurnStart = env.roundTrun[winIndex]
print('game end')
print()

예찬's first card is ['H', 7]
윤희's choise is ['H', 5]
상훈's choise is ['S', 13]
길용's choise is ['C', 14]
광호's choise is ['S', 6]
player 길용 win final round
game end



In [31]:
#게임 끝 승리팀 선언!!!
declarerScore, defenderScore = gf.finalWin(env.scores, env.role)
if declarerScore >= env.bidding:
    print('Declarer Win! 여당 승리!')
else:
    print('Defender Win! 야당 승리!')
print(declarerScore)
print(env.bidding)

Declarer Win! 여당 승리!
12
12


In [32]:
grave

[['H', 4], ['H', 6], ['D', 3]]

In [36]:
for i in range(5):
#     print(players[i].grave)
    print('I remember this grave ' + str(len(players[i].grave)))
    print('score is '+ str(players[i].score))
    print(len(players[i].hand))
    print(players[i].name + "' role is " + str(players[i].role))
#     print(env.scores[env.locations.index(env.roundTrun[i])])
    print()
    
print('player #:\t' + str(env.locations))
print('role:    \t' + str(env.role))
print('scores:  \t' + str(env.scores))
print('Bidding was\t' + str(env.bidding))

I remember this grave 50
score is 2
0
윤희' role is [0, 0]

I remember this grave 50
score is 8
0
상훈' role is [0, 1]

I remember this grave 50
score is 6
0
예찬' role is [0, 0]

I remember this grave 50
score is 0
0
광호' role is [0, 0]

I remember this grave 53
score is 4
0
길용' role is [1, 0]

player #:	[2, 0, 1, 4, 3]
role:    	[0, 0, 1, 1, 0]
scores:  	[6, 2, 8, 4, 0]
Bidding was	12


## 함수 연습 및 메모

In [6]:
# how to use (str).join
a = ['a', 'b', 'c', 'd']
a = [a]*4
for i in range(len(a)):
    print("...".join(a[i]))

a...b...c...d
a...b...c...d
a...b...c...d
a...b...c...d


In [11]:
b = 0
while b < 10:
    print(b)
    if b == 9:
        pass
#         break
    b += 1
else:
    print('This is "else"')
    print('and this is b: ' + str(b))
    print('If the loop exits as the result of a break, the else will not be executed.')

0
1
2
3
4
5
6
7
8
9
This is "else"
and this is b: 10
If the loop exits as the result of a break, the else will not be executed.


In [13]:
print("{} and {}".format("a", 'b'))

a and b


In [469]:
#핸드에서 카드 낼 때 list.() 쓰면 될듯

In [3]:
import re, random

with open('dict.txt', 'rt', encoding='utf-8') as f:
    s = f.read()

pat = re.compile('^[ㄱ-ㅎ가-힣]+$')
wordDict = dict()
hanbangSet = set()

# 한글로만 이루어져있고, 길이가 2 이상인 단어만 추출
for i in sorted([i for i in s.split() if pat.match(i) and len(i) >= 2], key=lambda x:-len(x)):
    if i[0] not in wordDict:
        wordDict[i[0]] = set()
    wordDict[i[0]].add(i)

# 한방단어 제거 & 추출
delList = list()
for i in wordDict:
    for j in wordDict[i]:
        if j[-1] not in wordDict:
            delList.append(j)
for j in delList:
    hanbangSet.add(j)
    wordDict[j[0]].remove(j)

print('\n 초고수 끝말잇기 ver.0.1')
print(' Ctrl+Z를 입력하면 기권할 수 있습니다.')
round, win, lose = 0, 0, 0

while True:
    # 라운드 시작
    round += 1
    print("\n" + "-" * 50)
    print("\n %d라운드를 시작합니다. 현재 %d승 %d패" % (round, win, lose))
    lastWord = ''
    alreadySet = set()
    firstTurn = True
    resetRound = False

    while True:
        # CPU 턴
        print()
        if firstTurn:
            lastWord = random.choice(list(wordDict[random.choice(list(wordDict.keys()))]))
            alreadySet.add(lastWord)
            print(' CPU : ' + lastWord)
            firstTurn = False
        else:
            firstLetter = lastWord[-1]
            if not list(filter(lambda x: x not in alreadySet, wordDict.get(firstLetter, set()))):
                # 라운드 종료
                print(' CPU : ^Z')
                print('\n [결과] CPU가 기권했습니다. 당신의 승리입니다!')
                win += 1
                break
            else:
                nextWords = sorted(filter(lambda x: x not in alreadySet, wordDict[firstLetter]), key=lambda x:-len(x))[:random.randint(20, 50)]
                lastWord = nextWords[random.randint(0, random.randrange(0, len(nextWords)))]
                alreadySet.add(lastWord)
                print(' CPU : ' + lastWord)

        # 유저 턴
        while True:
            print()
            try:
                yourWord = input(' YOU : ')
            except:
                print('\n [결과] 당신은 기권했습니다. CPU의 승리입니다!')
                print(' [힌트] ', end='')
                print(', '.join(list(filter(lambda x: x not in alreadySet, wordDict.get(lastWord[-1], set())))[:3]))
                resetRound = True
                lose += 1
                break
            firstLetter = yourWord[0]
            if firstLetter != lastWord[-1]:
                print(" [오류] '" + lastWord[-1] + "' (으)로 시작하는 단어를 입력하세요.")
            elif yourWord in hanbangSet:
                print(' [오류] 한방단어는 사용할 수 없습니다.')
            elif yourWord in alreadySet:
                print(' [오류] 이미 나온 단어입니다.')
            elif yourWord not in wordDict.get(firstLetter, set()):
                print(' [오류] 사전에 없는 단어입니다.')
            else:
                alreadySet.add(yourWord)
                lastWord = yourWord
                break

        if resetRound:
            # 라운드 종료
            break


 초고수 끝말잇기 ver.0.1
 Ctrl+Z를 입력하면 기권할 수 있습니다.

--------------------------------------------------

 1라운드를 시작합니다. 현재 0승 0패

 CPU : 령의정

 YOU : 정령

 CPU : 령출력원자로

 YOU : 로키

 CPU : 키카데오이데아

 YOU : 아시아

 CPU : 아말감농축폴라로그라프법

 YOU : 법학도

 CPU : 도급로동임금형태

 YOU : 태환

 CPU : 환경미화원

 YOU : 원자로

 CPU : 로맨티시스트

 YOU : 트롤

 CPU : 롤파쇄기

 YOU : 기창

 CPU : 창자막힘증

 YOU : 증축

 CPU : 축산작업반

 YOU : 반추

 CPU : 추종전동장치

 YOU : 치골

 CPU : 골목골목이

 YOU : 이재용
 [오류] 사전에 없는 단어입니다.

 YOU : 이빨

 CPU : 빨긋빨긋이

 YOU : 이빨
 [오류] 이미 나온 단어입니다.

 YOU : 이명

 CPU : 명충살이고치벌

 YOU : 벌레

 CPU : 레조디칼라브리아

 YOU : 아밀라아제

 CPU : 제비꼬리촉무음

 YOU : 음계

 CPU : 계미진신풍우록

 YOU : 록키
 [오류] 사전에 없는 단어입니다.

 YOU : 록음악
 [오류] 사전에 없는 단어입니다.

 YOU : 록
 [오류] 사전에 없는 단어입니다.

 YOU : 록록
 [오류] 사전에 없는 단어입니다.

 YOU : 녹녹
 [오류] '록' (으)로 시작하는 단어를 입력하세요.

 YOU : 록비

 CPU : 비프로레타리아

 YOU : 아비

 CPU : 비에니아프스키

 YOU : 키비

 CPU : 비자루바다지렁이

 YOU : 


IndexError: string index out of range

In [ ]:
import sys, os
import pandas as pd
import numpy as np
import json
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.display import display
from IPython.core.display import HTML

import seaborn as sns
from scipy import stats
import matplotlib as mpl

# import tensorflow as tf